In [2]:
import numpy as np
import keras as K
import tensorflow as tf
import os
import pandas as pd
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np

In [3]:
np.random.seed(1)
print(tf.__version__)

df = pd.read_csv("Gungor_2018_VictorianAuthorAttribution_data-train.csv")

df['author'].unique()

1.14.0


array([ 1,  2,  3,  4,  6,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37,
       38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 50])

In [4]:
p = np.random.permutation(df['author'].unique())
trainclass = p[:40]
fewclass = p[40:]

trainingclasses = df[df['author'].isin(trainclass)]

fewclasses = df[df['author'].isin(fewclass)]


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential, Model

from keras.layers import *
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional


os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

In [6]:
x = np.array(trainingclasses['text'])
y = np.array(pd.factorize(trainingclasses['author'])[0])

In [7]:
x.shape

(48472,)

In [10]:
import pickle

max_words = 20000
                
tokenizer = Tokenizer(20000)
tokenizer.fit_on_texts(x)
sequences = tokenizer.texts_to_sequences(x)

train_x, test_x, train_y, test_y = train_test_split(sequences, y, test_size=0.33, random_state=42)
max_review_len = 1000
train_x = K.preprocessing.sequence.pad_sequences(train_x,
  truncating='pre', padding='pre', maxlen=max_review_len)  # pad and chop!
test_x = K.preprocessing.sequence.pad_sequences(test_x,
  truncating='pre', padding='pre', maxlen=max_review_len)

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle)
    
word_index = tokenizer.word_index

In [18]:
embeddings_index = {}
f = open(os.path.join('./GLOVE_DIR', 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [19]:
embedding_matrix = np.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [22]:
embedding_layer = Embedding(len(word_index) + 1,
                            50,
                            weights=[embedding_matrix],
                            input_length=max_review_len,
                            trainable=False)

In [23]:
train_x, test_x, train_y, test_y = train_test_split(sequences,
                                                    y, test_size=0.33,
                                                    random_state=42)


In [71]:
input1 = Input(shape=(1000,))

x = embedding_layer(input1)

x = Conv1D(16, 3, strides=1)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)

x = Conv1D(16, 3, strides=2)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)

x = Conv1D(16, 3, strides=3)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)

x = Conv1D(16, 3, strides=3)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)

x = Conv1D(16, 3, strides=3)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)

feats = Flatten(name='features')(x)

x = Dense(40)(feats)
x = BatchNormalization()(x)
x = Activation('softmax', name='main_output')(x)

model = Model(inputs=input1, outputs=[x, feats])


print(model.summary()) 

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 1000)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 1000, 50)          500050    
_________________________________________________________________
conv1d_60 (Conv1D)           (None, 998, 16)           2416      
_________________________________________________________________
batch_normalization_73 (Batc (None, 998, 16)           64        
_________________________________________________________________
activation_60 (Activation)   (None, 998, 16)           0         
_________________________________________________________________
dropout_55 (Dropout)         (None, 998, 16)           0         
_________________________________________________________________
conv1d_61 (Conv1D)           (None, 498, 16)           784

In [80]:
opt = K.optimizers.Adam(lr=0.000001)
model.compile(opt, loss={'main_output': 'sparse_categorical_crossentropy'}, metrics=['acc'])


/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/keras/engine/training_utils.py:819: UserWarning: Output features missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to features.
  'be expecting any data to be passed to {0}.'.format(name))


In [81]:
# 3. train model
bat_size = 512
max_epochs = 50
print("\nStarting training ")
#model.fit(np.array(train_x), train_y, epochs=max_epochs,
#  batch_size=bat_size, shuffle=True, verbose=1) 
model.fit(np.array(train_x), train_y, epochs=max_epochs,
  batch_size=bat_size, shuffle=True, verbose=1, validation_data=(np.array(test_x), test_y)) 
print("Training complete \n")


Starting training 
Train on 32476 samples, validate on 15996 samples
Epoch 1/50
32476/32476 [==============================] - 5s 162us/step - loss: 1.7053 - main_output_loss: 1.7051 - main_output_acc: 0.5166 - val_loss: 1.7982 - val_main_output_loss: 1.7973 - val_main_output_acc: 0.5021
Epoch 2/50
32476/32476 [==============================] - 3s 92us/step - loss: 1.7052 - main_output_loss: 1.7055 - main_output_acc: 0.5187 - val_loss: 1.7976 - val_main_output_loss: 1.7967 - val_main_output_acc: 0.5018
Epoch 3/50
32476/32476 [==============================] - 3s 93us/step - loss: 1.7056 - main_output_loss: 1.7052 - main_output_acc: 0.5147 - val_loss: 1.7977 - val_main_output_loss: 1.7968 - val_main_output_acc: 0.5023
Epoch 4/50
32476/32476 [==============================] - 3s 93us/step - loss: 1.7018 - main_output_loss: 1.7021 - main_output_acc: 0.5178 - val_loss: 1.7973 - val_main_output_loss: 1.7964 - val_main_output_acc: 0.5021
Epoch 5/50
32476/32476 [=============================

Epoch 38/50
32476/32476 [==============================] - 3s 92us/step - loss: 1.6944 - main_output_loss: 1.6951 - main_output_acc: 0.5199 - val_loss: 1.7903 - val_main_output_loss: 1.7894 - val_main_output_acc: 0.5027
Epoch 39/50
32476/32476 [==============================] - 3s 92us/step - loss: 1.6930 - main_output_loss: 1.6940 - main_output_acc: 0.5192 - val_loss: 1.7900 - val_main_output_loss: 1.7892 - val_main_output_acc: 0.5028
Epoch 40/50
32476/32476 [==============================] - 3s 92us/step - loss: 1.6977 - main_output_loss: 1.6967 - main_output_acc: 0.5196 - val_loss: 1.7898 - val_main_output_loss: 1.7889 - val_main_output_acc: 0.5026
Epoch 41/50
32476/32476 [==============================] - 3s 92us/step - loss: 1.6929 - main_output_loss: 1.6931 - main_output_acc: 0.5189 - val_loss: 1.7902 - val_main_output_loss: 1.7894 - val_main_output_acc: 0.5028
Epoch 42/50
32476/32476 [==============================] - 3s 92us/step - loss: 1.6898 - main_output_loss: 1.6899 - main

In [ ]:
x2 = np.array(fewclasses['text'])
y2 = np.array(pd.factorize(fewclasses['author'])[0])

In [135]:
from keras.models import load_model

model = load_model('model_fewshot_1.h5')

/home/tokyotechies/miniconda3/envs/author_classification/lib/python3.6/site-packages/keras/engine/training_utils.py:819: UserWarning: Output features missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to features.
  'be expecting any data to be passed to {0}.'.format(name))


In [137]:
x3 = []

for i in range(5):
    x3.append(np.random.choice(x2[y2==i]))

In [136]:
print(train_x.shape)
print(x2.shape)
print(x3.shape)

(5, 1000)
(5206,)
(5,)


In [138]:
x3 = np.array(x3)

In [139]:
sequences = tokenizer.texts_to_sequences(x3)
test_x3 = tokenizer.texts_to_sequences(x2)



max_review_len = 1000
train_x = K.preprocessing.sequence.pad_sequences(sequences,
  truncating='pre', padding='pre', maxlen=max_review_len)  # pad and chop!
test_x = K.preprocessing.sequence.pad_sequences(test_x3,
  truncating='pre', padding='pre', maxlen=max_review_len)

In [140]:
train_features, _ = model.predict(train_x)

In [141]:
test_features, _ = model.predict(np.array(test_x3))

In [142]:
train_features = np.expand_dims(train_features, axis=0)
test_features = np.expand_dims(test_features, axis=1)

In [143]:
print(train_features.shape)
print(test_features.shape)

(1, 5, 40)
(5206, 1, 40)


In [144]:
dif = train_features - test_features

In [145]:
print(dif.shape)

(5206, 5, 40)


In [146]:


distance = np.linalg.norm(dif, axis=2)
predictions = np.argmin(distance, axis=1)
'''index = np.argpartition(distance, self.k)
values = index[:self.k]
unique, counts = np.unique(self.y[values], return_counts=True)
prediction = unique[np.argmax(counts)]'''
print(predictions.shape)

(5206,)


In [147]:
np.sum(predictions == y2)/5206

0.19996158278908951

In [50]:
model.save('model_fewshot_1.h5')

In [151]:
min(y2)

0